# More Data Preprocessing (with Pandas)

Pandas is a very useful data analytics package within Python.

Let's start with some random data.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame(np.random.randn(5, 3), index=['a', 'c', 'e', 'f', 'h'], columns=['one', 'two', 'three'])
df

,one,two,three
a,1.169242,-1.170645,-1.107838
c,0.806084,-1.120802,0.293680
e,-1.233944,1.538099,0.481668
f,-0.561147,0.549736,-0.869029
h,-0.573901,-0.407592,0.071266


We know with Pandas we can mix & match data types, so let us add two more columns, named 'four' and 'five':

In [3]:
df['four'] = 'bar'
df['five'] = df['one'] > 0
df

,one,two,three,four,five
a,1.169242,-1.170645,-1.107838,bar,True
c,0.806084,-1.120802,0.293680,bar,True
e,-1.233944,1.538099,0.481668,bar,False
f,-0.561147,0.549736,-0.869029,bar,False
h,-0.573901,-0.407592,0.071266,bar,False


Adding new rows is also simple. Below we include three extra empty rows:

In [4]:
df2 = df.reindex(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])
df2

,one,two,three,four,five
a,1.169242,-1.170645,-1.107838,bar,True
b,NaN,NaN,NaN,NaN,NaN
c,0.806084,-1.120802,0.293680,bar,True
d,NaN,NaN,NaN,NaN,NaN
e,-1.233944,1.538099,0.481668,bar,False
f,-0.561147,0.549736,-0.869029,bar,False
g,NaN,NaN,NaN,NaN,NaN
h,-0.573901,-0.407592,0.071266,bar,False


# Your Turn Here

Do you still remember how to index a row/column?

In [6]:
#### index row 'c' below
df2.loc['c']

one      0.806084
two       -1.1208
three     0.29368
four          bar
five         True
Name: c, dtype: object

In [7]:
#### index column 'two' below
df2['two']

a   -1.170645
b         NaN
c   -1.120802
d         NaN
e    1.538099
f    0.549736
g         NaN
h   -0.407592
Name: two, dtype: float64

pandas has two functions isnull() and notnull() that return boolean objects when called.

In [8]:
pd.isnull(df2['one'])

a    False
b     True
c    False
d     True
e    False
f    False
g     True
h    False
Name: one, dtype: bool

In [9]:
pd.notnull(df2['one'])

a     True
b    False
c     True
d    False
e     True
f     True
g    False
h     True
Name: one, dtype: bool

Missing values propagate naturally through arithmetic operations between pandas objects.

In [10]:
a = df[['one','two']]
a['one']['a':'e'] = float('nan')
a

/opt/tljh/user/lib/python3.6/site-packages/pandas/core/series.py:1240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
/opt/tljh/user/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,one,two
a,NaN,-1.170645
c,NaN,-1.120802
e,NaN,1.538099
f,-0.561147,0.549736
h,-0.573901,-0.407592


In [11]:
b = df[['one','two','three']]
b

,one,two,three
a,1.169242,-1.170645,-1.107838
c,0.806084,-1.120802,0.293680
e,-1.233944,1.538099,0.481668
f,-0.561147,0.549736,-0.869029
h,-0.573901,-0.407592,0.071266


In [12]:
a + b

,one,three,two
a,NaN,NaN,-2.341290
c,NaN,NaN,-2.241603
e,NaN,NaN,3.076198
f,-1.122294,NaN,1.099472
h,-1.147801,NaN,-0.815184


# How to deal with Missing Values

## Deleting Missing Values

The simplest method is always dropping all missing values - but it is highly **discouraged!!!**

In [13]:
a['one'].dropna()

f   -0.561147
h   -0.573901
Name: one, dtype: float64

By default, dropna() will drop any row containing **NaN** values, but you can change that by using the *axis=* and *thresh=* arguments.

**NOTE**: Dropping rows or columns have different uses.

In [14]:
#### This statement drop any column with NaN values
a.dropna(axis=1)

,two
a,-1.170645
c,-1.120802
e,1.538099
f,0.549736
h,-0.407592


In [15]:
#### thresh determines how many non-NaN values a column/row should have without being dropped
c = a + b
c.dropna(axis=1, thresh=2)

,one,two
a,NaN,-2.341290
c,NaN,-2.241603
e,NaN,3.076198
f,-1.122294,1.099472
h,-1.147801,-0.815184


You can also use the *how=* argument to determine how do you want to remove the NaN values.

In [16]:
#### By default, dropna() drops column/row with any NaN values
#### how = 'all' changes that to dropping column/row with all NaN values
c.dropna(axis=1, how='all')

,one,two
a,NaN,-2.341290
c,NaN,-2.241603
e,NaN,3.076198
f,-1.122294,1.099472
h,-1.147801,-0.815184


You can refer to the [pandas.dropna() docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html) for more information.

## Imputing Missing Values

Imputing means filling missing values - you can do that when the missing and non-missing values are in some type of relationship.

In [17]:
my_series = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
my_series

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [18]:
#### you can fill missing values with a specific value (0)
my_series.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Alternatively, we can specify a forward-fill to propagate the previous value forward:

In [19]:
# forward-fill
my_series.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Or we can specify a back-fill to propagate the next values backward:

In [20]:
my_series.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

## Your Turn Here
Aforementioned method can be applied to dataframes. 

In [21]:
#### Let us generate a random dataframe
rand_df = pd.DataFrame(np.random.randn(5, 3), 
                  index=['a', 'b', 'c', 'd', 'e'],
                  columns=['one', 'two', 'three'])
rand_df = rand_df.mask(np.random.random(rand_df.shape) < .3)
rand_df

,one,two,three
a,0.178649,1.321487,NaN
b,0.811175,NaN,0.457116
c,1.096891,-0.493531,0.125251
d,0.364773,0.399025,1.130262
e,0.728406,0.988974,-1.739564


Your tasks are as follows:

- fill missing values in column **one** with value 1;
- fill missing values in column **two** with forward-filling;
- fill missing values in column **three** with backward-filling.

In [23]:
#### insert your code here
rand_df['one'] = rand_df['one'].fillna(1)
rand_df['two'] = rand_df['two'].fillna(method = 'ffill')
rand_df['three'] = rand_df['three'].fillna(method = 'bfill')
rand_df

,one,two,three
a,0.178649,1.321487,0.457116
b,0.811175,1.321487,0.457116
c,1.096891,-0.493531,0.125251
d,0.364773,0.399025,1.130262
e,0.728406,0.988974,-1.739564


A useful approach for imputing your missing data is to use mean/mode to replace missing data - the reason behind this logic is that if we are going to *guess* the values of the missing data, the highest chance would be guessing it to be the mean/mode if the data follows **normal** distribution.

In [24]:
#### let us generate another DF
my_df = pd.DataFrame(np.random.randn(5, 3), 
                  index=['a', 'b', 'c', 'd', 'e'],
                  columns=['A', 'B', 'C'])
my_df = my_df.mask(np.random.random(my_df.shape) < .3)
my_df

,A,B,C
a,NaN,NaN,2.105091
b,0.530084,NaN,-1.283248
c,NaN,2.237358,0.599867
d,NaN,0.378969,-1.211894
e,1.331069,0.366033,NaN


In [25]:
#### Let us check if there is any missing value in the df
my_df.isnull().values.any()

True

In [26]:
#### Then we are getting the mean of the DF
#### Note that since we only 
my_df.mean()

A    0.930576
B    0.994120
C    0.052454
dtype: float64

In [27]:
df_filled = my_df.fillna(my_df.mean())
df_filled

,A,B,C
a,0.930576,0.994120,2.105091
b,0.530084,0.994120,-1.283248
c,0.930576,2.237358,0.599867
d,0.930576,0.378969,-1.211894
e,1.331069,0.366033,0.052454


In [28]:
#### Now let us check again if there is any missing values
df_filled.isnull().values.any()

False

More info regarding how to handle missing data can be found [here](https://machinelearningmastery.com/handle-missing-data-python/).

# Other Tasks in Data Preprocessing

- Handling categorical data (coding)
- Handling imbalanced data
- feature engineering

These topics will be covered in later part of this class.